# Model with Metrics

Example testing a model with custom metrics.

Metrics can be 

  * A ```COUNTER``` : the returned value will increment the current value
  * A ```GAUGE``` : the returned value will overwrite the current value
  * A ```TIMER``` : a number of millisecs. Prometheus SUM and COUNT metrics will be created.
  
You need to provide a list of dictionaries each with the following:

  * a ```type``` : COUNTER, GAUGE, or TIMER
  * a ```key``` : a user defined key
  * a ```value``` : a float value
  
See example code below:
 

In [ ]:
!pygmentize ModelWithMetrics.py

# REST

In [ ]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.4 model-with-metrics-rest:0.1

In [ ]:
!docker run --name "model-with-metrics" -d --rm -p 5000:5000 model-with-metrics-rest:0.1

In [ ]:
!cd ../../../wrappers/testing && make build_protos

## Test predict

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p

In [ ]:
!docker rm model-with-metrics --force

# gRPC

In [ ]:
!s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.4 model-with-metrics-grpc:0.1

In [ ]:
!docker run --name "model-with-metrics" -d --rm -p 5000:5000 model-with-metrics-grpc:0.1

In [ ]:
!cd ../../../wrappers/testing && make build_protos

## Test predict

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --grpc

In [ ]:
!docker rm model-with-metrics --force

# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

In [ ]:
!helm install seldon-core-analytics --name seldon-core-analytics --set grafana_prom_admin_password=password --set persistence.enabled=false --repo https://storage.googleapis.com/seldon-charts 

 * Port forward the dashboard when running
  ```
     kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
  ```
  * Visit http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1 and login using "admin" and the password you set above when launching with helm.

# REST

In [ ]:
!eval $(minikube docker-env) && s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.4 model-with-metrics-rest:0.1 --loglevel 5

In [ ]:
!kubectl create -f deployment-rest.json

Wait until ready (replicas == replicasAvailable)

In [ ]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

In [ ]:
!cd ../../../util/api_tester && make build_protos 

## Test predict

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!kubectl delete -f deployment-rest.json

# gRPC

In [ ]:
!eval $(minikube docker-env) && s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.4 model-with-metrics-grpc:0.1

In [ ]:
!kubectl create -f deployment-grpc.json

Wait until ready (replicas == replicasAvailable)

In [ ]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

In [ ]:
!cd ../../../util/api_tester && make build_protos 

## Validate on Grafana

To check the metrics have appeared on Prometheus and are available in Grafana you could create a new graph in a dashboard and use the query:

```
mycounter_total
```


## Test predict

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --grpc --oauth-port `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'`

In [ ]:
!kubectl delete -f deployment-grpc.json

In [ ]:
!minikube delete